#Proyecto Individual N°1 - Gaby Goñalons

#Se importan las librerías a utilizar

In [35]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime

#Utilizando Pandas, se leen los datos y se visualizan

In [4]:
# Se lee el archivo CSV especificando el tipo de datos de la columna overview como cadena de texto
movies = pd.read_csv("../ETL_API/movies_dataset.csv", dtype=str)
movies['overview'] = movies['overview'].astype(str)
movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92


#Tamaño del Dataframe

In [5]:
filas = movies.shape[0]
columnas = movies.shape[1]
print("El DataFrame posee ", filas, "filas y ", columnas, "columnas")


El DataFrame posee  45466 filas y  24 columnas


#Se eliminan las columnas: video, imdb_id, adult, original_title, poster_path y home_page.

In [6]:
#Se eliminan las columnas indicadas
movies = movies.drop(columns=['adult', 'homepage', 'imdb_id', 'original_title', 'poster_path', 'video'])

#Se calcula la cantidad de filas/columnas
movies.shape

(45466, 18)

#Se trabaja con las columnas revenue y budget

In [7]:
#Se calcula la cantidad de valores nulos en la columna "revenue"
num_revenue_null = movies["revenue"].isnull().sum()

#Se calcula la cantidad de valores nulos en la columna "budget"
num_budget_null = movies["budget"].isnull().sum()

#Se imprimen los valores
print(f"La columna 'revenue' tiene {num_revenue_null} valores nulos.")
print(f"La columna 'budget' tiene {num_budget_null} valores nulos.")

La columna 'revenue' tiene 6 valores nulos.
La columna 'budget' tiene 0 valores nulos.


In [8]:
#Se remplazan los valores nulos por ceros en los campos revenue y budget
movies["revenue"].fillna(0, inplace=True)
movies["budget"].fillna(0, inplace=True)

# Se calcula la cantidad de valores nulos en la columna "revenue"
num_revenue_null = movies["revenue"].isnull().sum()

#Se calcula la cantidad de valores nulos en la columna "budget"
num_budget_null = movies["budget"].isnull().sum()

#Se imprimen los valores
print(f"La columna 'revenue' tiene {num_revenue_null} valores nulos.")
print(f"La columna 'budget' tiene {num_budget_null} valores nulos.")

La columna 'revenue' tiene 0 valores nulos.
La columna 'budget' tiene 0 valores nulos.


#Se trabajo con la columna release_date

In [9]:
# Se imprime la información del dataframe en relación a los tipos de datos
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   belongs_to_collection  4494 non-null   object
 1   budget                 45466 non-null  object
 2   genres                 45466 non-null  object
 3   id                     45466 non-null  object
 4   original_language      45455 non-null  object
 5   overview               45466 non-null  object
 6   popularity             45461 non-null  object
 7   production_companies   45463 non-null  object
 8   production_countries   45463 non-null  object
 9   release_date           45379 non-null  object
 10  revenue                45466 non-null  object
 11  runtime                45203 non-null  object
 12  spoken_languages       45460 non-null  object
 13  status                 45379 non-null  object
 14  tagline                20412 non-null  object
 15  title              

#Se extre el año del campo release_date y se coloca en una nueva columna release_year

In [15]:
# Se convierte de la columna 'release_date' a datetime y se extrae del año
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
movies['release_year'] = movies['release_date'].dt.year.fillna(0).astype(int)

# Imprimir la columna 'release_year'
print(movies['release_year'])

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45461       0
45462    2011
45463    2003
45464    1917
45465    2017
Name: release_year, Length: 45466, dtype: int32


#Se calculo la columna retorno como el cociente entre revenue y budget

In [16]:
# Se convierten los campos a numéricos
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce')
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')

# Se calcula el cociente y se asigna a la columna return redondeando a dos decimales el valor resultante
return_values = movies['revenue'].fillna(0) / movies['budget'].fillna(0)
return_values = return_values.replace([np.inf, -np.inf], np.nan).fillna(0)
movies['return'] = round(return_values, 2)

# Se imprime la serie return
print(movies['return'])


0        12.45
1         4.04
2         0.00
3         5.09
4         0.00
         ...  
45461     0.00
45462     0.00
45463     0.00
45464     0.00
45465     0.00
Name: return, Length: 45466, dtype: float64


##Transformaciones:
###Campos Anidados - se desanidaron los campos utilizando la librería ast.-

1 - belongs_to_collection: Se trata de una lista de diccionarios, que tambíen posee valores nulos.

In [17]:
# Función para obtener el nombre de la colección
def get_collection_name(collection):
    if pd.notnull(collection):
        try:
            collection_values = ast.literal_eval(collection)
            if isinstance(collection_values, dict):
                return collection_values.get('name')
        except (SyntaxError, ValueError):
            pass
    return ''

# Se aplica la función a la columna 'belongs_to_collection'
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(get_collection_name)

# Se muestran las primeras 5 filas del DataFrame con los datos desanidados en la columna 'belongs_to_collection'
movies['belongs_to_collection'].head(5)


0              Toy Story Collection
1                                  
2         Grumpy Old Men Collection
3                                  
4    Father of the Bride Collection
Name: belongs_to_collection, dtype: object

2 - genres: Se trata de una lista de diccionarios, que tambíen posee valores nulos.

In [18]:
# Función para obtener los nombres de los géneros
def get_genre_names(genres):
    genre_names = []
    if pd.notnull(genres):
        try:
            genre_values = ast.literal_eval(genres)
            if isinstance(genre_values, list):
                genre_names = [genre['name'] for genre in genre_values]
        except (SyntaxError, ValueError):
            pass
    return genre_names

# Se aplica la función a la columna 'genres'
movies['genres'] = movies['genres'].apply(get_genre_names)

# Se muestran las primeras 5 filas del DataFrame con los datos desanidados en la columna 'genres'
movies['genres'].head(5)


0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

3 - production_companies: Se trata de una lista de diccionarios, que tambíen posee valores nulos.

In [19]:
# Función para obtener el nombre de la productora
def get_production_company_name(production_companies):
    if pd.notnull(production_companies):
        try:
            company_values = ast.literal_eval(production_companies)
            if isinstance(company_values, list) and len(company_values) > 0:
                return company_values[0].get('name')
        except (SyntaxError, ValueError):
            pass
    return ''

# Se aplica la función a la columna 'production_companies'
movies['production_companies'] = movies['production_companies'].apply(get_production_company_name)

# Se muestran las primeras 5 filas del DataFrame con los datos desanidados en la columna 'production_companies'
movies['production_companies'].head(5)


0                   Pixar Animation Studios
1                          TriStar Pictures
2                              Warner Bros.
3    Twentieth Century Fox Film Corporation
4                     Sandollar Productions
Name: production_companies, dtype: object

4 - spoken_languages: Se trata de una lista de diccionarios, que tambíen posee valores nulos.

In [20]:
# Función para obtener el nombre del idioma
def get_language_name(spoken_languages):
    if pd.notnull(spoken_languages):
        try:
            language_values = ast.literal_eval(spoken_languages)
            if isinstance(language_values, list) and len(language_values) > 0:
                return language_values[0].get('name')
        except (SyntaxError, ValueError):
            pass
    return ''

# Se aplica la función a la columna 'spoken_languages'
movies['spoken_languages'] = movies['spoken_languages'].apply(get_language_name)

# Se muestran las primeras 5 filas del DataFrame con los datos desanidados en la columna 'spoken_languages'
movies['spoken_languages'].head(5)


0    English
1    English
2    English
3    English
4    English
Name: spoken_languages, dtype: object

#Se visualiza el dataframe

In [21]:
movies.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,English,Released,NaN,Toy Story,7.7,5415,1995,12.45
1,,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,English,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413,1995,4.04
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,Warner Bros.,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92,1995,0.00


#Se lee el archhivo credits.csv

In [23]:
# Se lee el archivo CSV especificando el tipo de datos de la columna overview como cadena de texto
credits = pd.read_csv("../ETL_API/credits.csv", dtype=str)
credits.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


#Se extraen los Actores del campo cast

In [24]:
# Se desanida el campo 'cast' y  se crear una nueva columna 'actors'
def extract_actors(cast):
    actors = []
    if pd.notnull(cast):
        try:
            cast_values = ast.literal_eval(cast)
            actors = [item['name'] for item in cast_values]
        except (SyntaxError, ValueError):
            pass
    return actors

movies['actors'] = credits['cast'].apply(extract_actors)

# Se  muestran las primeras 3 filas del DataFrame 'movies' con la columna 'actors'
movies['actors'].head(3)


0    [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...
1    [Robin Williams, Jonathan Hyde, Kirsten Dunst,...
2    [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...
Name: actors, dtype: object

In [25]:
# Se desanida el campo 'crew' y se crea una nueva columna 'directors'
def extract_directors(crew):
    directors = []
    if pd.notnull(crew):
        try:
            crew_values = ast.literal_eval(crew)
            directors = [item['name'] for item in crew_values if item['job'] == 'Director']
        except (SyntaxError, ValueError):
            pass
    return directors

movies['directors'] = credits['crew'].apply(extract_directors)

# Se muestran las primeras 5 filas del DataFrame 'movies' con la columna 'directors'
movies['directors'].head(3)


0    [John Lasseter]
1     [Joe Johnston]
2    [Howard Deutch]
Name: directors, dtype: object

#Se crea el archivo del dataframe trabajado

In [26]:
movies.to_csv('movies_ETL.csv', index=False)

#Se toma del dataframe exclusivamente los campos a utilizar por las funciones de la API a efectos de optimizar el uso del almacenamiento. Se visualizan los campos y se analiza los requeridos.

In [28]:
movies.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,actors,directors
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,English,Released,NaN,Toy Story,7.7,5415,1995,12.45,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter]


#Se eliminan las columnas inncesarias

In [30]:
movies = movies.drop(columns=['belongs_to_collection', 'id', 'original_language', 'overview', 'production_companies', 'production_countries','spoken_languages','status','tagline'])

#Se muetra el dataframe resultante.

In [31]:
movies.head(1)

,budget,genres,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,actors,directors
0,30000000.0,"[Animation, Comedy, Family]",21.946943,1995-10-30,373554033,81.0,Toy Story,7.7,5415,1995,12.45,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter]


In [34]:
movies.shape

(45466, 13)

#Para evitar inconvenientes al momento de subir los archivos por las limitaciones de render se adopta el criterio de seleccionar las películas de los últimos 10 años.-

In [36]:
# Se convierte la columna "release_date" a tipo datetime
movies['release_date'] = pd.to_datetime(movies['release_date'])

# Se obtiene la fecha actual
current_date = datetime.now()

# Se calcula la fecha mínima para los últimos 10 años
ten_years_ago = current_date - pd.DateOffset(years=10)

# Se filtran los datos de los últimos 10 años
movies = movies[movies['release_date'] >= ten_years_ago]

# Se reinician los índices del DataFrame resultante
movies = movies.reset_index(drop=True)

movies.shape

(7133, 13)

#Se genera el archivo .csv a trabajar con la API

In [37]:
movies.to_csv('movies_API.csv', index=False)